### Machine-learning Model Evaluation for Mode of Delivery Predictions Using CDC Vital Statistics Data
Perform 10-fold cross-validation on training set using 3 different classifiers and increasing number of model parameters. Relies on filtered input data created by companion **deliveryMode** notebook.<br><br>
&copy; 2018-2022 Karl W. Schulz<br>
University of Texas<br>
***


In [1]:
import pickle
import utils
import ml_utils
import time
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.metrics import accuracy_score, f1_score, brier_score_loss
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.utils.class_weight import compute_sample_weight
timer = utils.timer()

#### Load filtered data from training set

In [2]:
file = open('data/filtered.training.pickle', 'rb')
data = pickle.load(file)
file.close()

### Define modeling parameters for $t_{early}$ (bin1) and $t_{term}$ scenarios (bin2)

In [3]:
params = {}
params['bin1'] = ['mager','lbo','tbo',
                 'rf_ppterm_bool','rf_cesar_bool','rf_cesarn','rf_diab_bool','rf_phyp_bool','rf_eclam_bool',
                 'cig_0','cig_1','cig_2','cig_3',
                 'bmi_r','pwgt_r',
                 'mbrace_1',  'mbrace_2',  'mbrace_3',  'mbrace_4', 'mbrace_5',
                 'mbrace_6',  'mbrace_7',  'mbrace_8',  'mbrace_9', 'mbrace_10',
                 'mbrace_11', 'mbrace_12', 'mbrace_13', 'mbrace_14','mbrace_15',
                 'umhisp_0',  'umhisp_1',  'umhisp_2',  'umhisp_3', 'umhisp_4', 'umhisp_5', 'umhisp_9',
             ]
params['bin2'] = params['bin1'] + ['rf_ghyp_bool','rf_gest_bool', 'previs_rec','combgest','wtgain_rec']

# define number of modeling subfeatures to consider
subFeatureCounts = [5,10,15,20]

# Define classification variable and weights
modelY = data.filter(items=['non_elective'])
weight = compute_sample_weight(class_weight='balanced', y=modelY)

# Define classification variable
qoi    = ['non_elective']
modelY = data.filter(items=['non_elective'])

# choose scoring metrics
metrics = ['roc_auc', 'accuracy','recall','f1','precision','brier_score_loss']

# define random seed
random_state=42

scores = {}

#### &rarr;  XGBoost Model Evaluation

In [4]:
clf = XGBClassifier(random_state=random_state,n_jobs=-1,tree_method='exact')

scores["xgboost-bin1"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin1',params,metrics,data,modelY,weight)
scores["xgboost-bin2"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin2',params,metrics,data,modelY,weight)


Running XGBClassifier for bin1 params
max # of params = 36
Total time to perform single train using all parameters = 100.832 (secs)
Sorting features based on SHAP importance

--> Running cross-validation for 5 features
Total time to perform cross-validation for 5 features = 655.427 (secs)

--> Running cross-validation for 10 features
Total time to perform cross-validation for 10 features = 782.193 (secs)

--> Running cross-validation for 15 features
Total time to perform cross-validation for 15 features = 769.239 (secs)

--> Running cross-validation for 20 features
Total time to perform cross-validation for 20 features = 825.485 (secs)

--> Running cross-validation for 36 features
Total time to perform cross-validation for all 36 features = 950.292 (secs)

                   Model       AUC          Accuracy         Recall            F1           Brier    
--------------------------------------------------------------------------------------------------------------
              XGB-b

#### &rarr;  AdaBoost Model Evaluation

In [5]:
clf = AdaBoostClassifier(random_state=random_state)
scores["adaboost-bin1"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin1',params,metrics,data,modelY,weight)
scores["adaboost-bin2"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin2',params,metrics,data,modelY,weight)


Running AdaBoostClassifier for bin1 params
max # of params = 36
Total time to perform single train using all parameters = 407.635 (secs)
Sorting features based on gini importance

--> Running cross-validation for 5 features
Total time to perform cross-validation for 5 features = 2010.527 (secs)

--> Running cross-validation for 10 features
Total time to perform cross-validation for 10 features = 2977.303 (secs)

--> Running cross-validation for 15 features
Total time to perform cross-validation for 15 features = 2480.266 (secs)

--> Running cross-validation for 20 features
Total time to perform cross-validation for 20 features = 2685.629 (secs)

--> Running cross-validation for 36 features
Total time to perform cross-validation for all 36 features = 3583.461 (secs)

                   Model       AUC          Accuracy         Recall            F1           Brier    
--------------------------------------------------------------------------------------------------------------
         

#### &rarr;  Complement Naive Bayes Model Evaluation
Note that this classifier does not support feature_importances directly. We pass an optional arg here (feature_log=True) to compute importance from feature_log_prob_

In [6]:
clf = ComplementNB()
scores["cnbayes-bin1"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin1',params,metrics,data,
                                                                    modelY,weight,feature_log=True)
scores["cnbayes-bin2"] = ml_utils.cross_validation_with_subfeatures(clf,subFeatureCounts,'bin2',params,metrics,data,
                                                                    modelY,weight,feature_log=True)


Running ComplementNB for bin1 params
max # of params = 36
Total time to perform single train using all parameters = 2.770 (secs)

--> Running cross-validation for 5 features
Total time to perform cross-validation for 5 features = 24.875 (secs)

--> Running cross-validation for 10 features
Total time to perform cross-validation for 10 features = 30.476 (secs)

--> Running cross-validation for 15 features
Total time to perform cross-validation for 15 features = 35.147 (secs)

--> Running cross-validation for 20 features
Total time to perform cross-validation for 20 features = 38.214 (secs)

--> Running cross-validation for 36 features
Total time to perform cross-validation for all 36 features = 58.186 (secs)

                   Model       AUC          Accuracy         Recall            F1           Brier    
--------------------------------------------------------------------------------------------------------------
     ComplementNB-bin1-5  0.661 +/- 0.018  0.578 +/- 0.028  0.663 +/-